In [1]:
import os, sys
sys.path.append('ext/deepFibreTracking/')

import numpy as np
import random
import gym
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import envs.RLtractEnvironment as RLTe
from src.data import HCPDataContainer, ISMRMDataContainer

# Initialise environment

In [2]:
RLenv = RLTe.RLtractEnvironment(device = 'cpu', grid_dim=[3,3,3])

Reset to streamline 128/5576


In [3]:
nextState, rewardNextState, done = RLenv.step(10)
print("Reward: %.3f" % (rewardNextState))

Reward: -0.004


In [4]:
nextState.getValue()

tensor([[[[0.0762, 0.1170, 0.1829],
          [0.1078, 0.1741, 0.2614],
          [0.1478, 0.2085, 0.2185]],

         [[0.0959, 0.1519, 0.2191],
          [0.1344, 0.2051, 0.2773],
          [0.1767, 0.2341, 0.2316]],

         [[0.1461, 0.2228, 0.2790],
          [0.1882, 0.2626, 0.3137],
          [0.2358, 0.2902, 0.2627]]],


        [[[0.0641, 0.1071, 0.1704],
          [0.0996, 0.1714, 0.2567],
          [0.1442, 0.2210, 0.2470]],

         [[0.0816, 0.1350, 0.1950],
          [0.1227, 0.1954, 0.2602],
          [0.1732, 0.2427, 0.2432]],

         [[0.1301, 0.2017, 0.2528],
          [0.1701, 0.2446, 0.2895],
          [0.2289, 0.2921, 0.2610]]],


        [[[0.0734, 0.1133, 0.1684],
          [0.1095, 0.1723, 0.2371],
          [0.1492, 0.2113, 0.2072]],

         [[0.0913, 0.1432, 0.1935],
          [0.1308, 0.1959, 0.2417],
          [0.1723, 0.2245, 0.2135]],

         [[0.1436, 0.2105, 0.2433],
          [0.1813, 0.2458, 0.2663],
          [0.2228, 0.2680, 0.2332]]],


    

# Tracker

In [16]:
from src.data import HCPDataContainer, ISMRMDataContainer
from src.data.postprocessing import res100
from src.tracker import DTITracker, CSDTracker, ISMRMReferenceStreamlinesTracker, StreamlinesFromFileTracker

In [17]:
hcp_sl = CSDTracker(RLenv.dataset, random_seeds=True, seeds_count=40000)
hcp_sl.track()

/home/hoffma83/.local/lib/python3.6/site-packages/fury/utils.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


In [18]:
sls = hcp_sl.get_streamlines()

In [19]:
streamline_index = np.random.randint(len(sls))
referenceStreamline = sls[streamline_index]

In [20]:
filtered_streamlines = hcp_sl.filtered_streamlines_by_length(minimum=40) # filter streamlines

hcp_sl.save_to_file("data/HCP307200_CSD_min40.vtk")

/home/hoffma83/.local/lib/python3.6/site-packages/fury/utils.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


In [21]:
len(filtered_streamlines)

23151

In [11]:
ref = torch.FloatTensor(referenceStreamline_ijk)

NameError: name 'referenceStreamline_ijk' is not defined

In [ ]:
ref.shape

In [ ]:
pts = torch.FloatTensor(np.array([50,50,50])).view(1,-1)
pts = ref[10]+10

In [ ]:
minDist = torch.min(torch.sum( (ref - pts)**2, dim =1 ))

In [ ]:
minDist

In [ ]:
torch.nn.functional.leaky_relu(-1 * minDist)

In [ ]:
torch.where(minDist < 0.1, 1,2)